In [1]:
from keras.layers import Bidirectional, CuDNNLSTM, Multiply,Dropout
from keras.layers import  Dense
from keras.utils import to_categorical
from keras.models import Sequential
from keras.callbacks import ModelCheckpoint
import numpy as np
import pickle
from sklearn.preprocessing import StandardScaler

C:\Users\djaym7\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
address = 'D:\\data_mfcc\\'

In [3]:
def loadData(id):
    with open(address + f'y_train_{id}','rb') as f:
        y_train=np.array(pickle.load(f))
    with open(address+f'y_test_{id}','rb') as f:
        y_test=np.array(pickle.load(f))
    with open(address+ f'X_train_{id}','rb') as f:
        X_train = np.array(pickle.load(f))
    with open(address+f'X_test_{id}','rb') as f:
        X_test = np.array(pickle.load(f))

    scalers = {}
    for k in range(X_train.shape[2]):
        scalers[k] = StandardScaler()
        X_train[:, k, :] = scalers[k].fit_transform(X_train[:, k, :]) 

    for j in range(X_test.shape[2]):
        X_test[:, j, :] = scalers[j].transform(X_test[:, j, :])
    
    print(f'id : {id}')
    from sklearn.preprocessing import LabelBinarizer
    encoder = LabelBinarizer()
    y_train = encoder.fit_transform(y_train)
    y_test = encoder.fit_transform(y_test)

   
    return X_train,y_train,X_test,y_test

In [ ]:
total_num_people=[]
for i in range(1,92):
    if i <10:
        total_num_people.append('100'+str(i))
    else:
        total_num_people.append('10'+str(i))

for i in total_num_people[15:59]:
    if int(i)<1078:
        X_train=[]
        X_test=[]
        y_train=[]
        y_test=[]
        X_train,y_train,X_test,y_test=loadData(i)
    else:
        continue

    time_steps = X_train.shape[1]
    input_dim = X_train.shape[2]
    model = Sequential()


    model.add(Bidirectional(CuDNNLSTM(16,input_shape=(time_steps,input_dim),return_sequences=True)))
    model.add(Dropout(0.4))
    model.add(CuDNNLSTM(96))
    model.add(Dense(6,activation='softmax'))
 

    
    model.compile(loss='categorical_crossentropy', optimizer= 'adam', metrics=['accuracy'])
    print(f'id : {i}')
    filePath =r"C:\Users\djaym7\Desktop\Github\EmotionRecognition\saved_bi_na_models"+ f'/NA{i}.h5'
    checkpoint = ModelCheckpoint(filepath=filePath,monitor='val_acc',mode='max',save_best_only=True,verbose=1,save_weights_only=False)
            
    model.fit(X_train, y_train,batch_size=100, epochs=150,validation_data=(X_test,y_test),callbacks=[checkpoint])
    

id : 1016
id : 1016
Train on 7360 samples, validate on 82 samples
Epoch 1/150
7360/7360 [==============================] - 29s 4ms/step - loss: 1.7950 - acc: 0.1689 - val_loss: 1.7958 - val_acc: 0.1585

Epoch 00001: val_acc improved from -inf to 0.15854, saving model to C:\Users\djaym7\Desktop\Github\EmotionRecognition\saved_bi_na_models/NA1016.h5
Epoch 2/150
7360/7360 [==============================] - 23s 3ms/step - loss: 1.7917 - acc: 0.1764 - val_loss: 1.7802 - val_acc: 0.1829

Epoch 00002: val_acc improved from 0.15854 to 0.18293, saving model to C:\Users\djaym7\Desktop\Github\EmotionRecognition\saved_bi_na_models/NA1016.h5
Epoch 3/150
7360/7360 [==============================] - 25s 3ms/step - loss: 1.7815 - acc: 0.1842 - val_loss: 1.7373 - val_acc: 0.2195

Epoch 00003: val_acc improved from 0.18293 to 0.21951, saving model to C:\Users\djaym7\Desktop\Github\EmotionRecognition\saved_bi_na_models/NA1016.h5
Epoch 4/150
7360/7360 [==============================] - 23s 3ms/step - loss